# Udělej si sám/sama: regulární výrazy

Jak jsme si říkali na jedné z předchozích hodin, regulární výrazy představují speciální jazyk pro vyhledávání vzorců v textových řetězcích. Díky nim můžeme jedním zadáním zachytit velké množství vzorců, které nás zajímají: např. regulární výraz `nej.*` znamená, že hledáme písmena `n`, `e` a `j` následovaná libovolným počtem (`*`) libovolných znaků (`.`).

V tomto úkolu se pomocí několika funkcí pokusíme vytvořit jednoduchý systém na matchování regulárních výrazů. Naprostou většinu jejich standardní funkcionality zanedbáme, zajímá nás vlastně jen jedna speciální dovednost: schopnost znaku `.` matchovat jakýkoli znak. Jinak náš systém bude vlastně fungovat podobně jako metoda `.startswith()` na řetězcích:

In [1]:
# výsledek je True a náš systém by měl taky vrátit True
"abcdef".startswith("abc")

True

In [2]:
# tohle vrátí False, ale náš systém vrátí True
"abcdef".startswith(".bc")

False

In [3]:
# tady náš systém vrátí taky False, protože zpětné lomítko
# \ speciální chování tečky "vypne" (tzv. *escaping*)
"abcdef".startswith(r"\.bc")

False

Pochopitelně toho musíme dosáhnout, **aniž bychom použili jakoukoli existující knihovnu na práci s regulárními výrazy** (`re`, `regex` apod.) :)

## Parsování regulárního výrazu

Skoro všechny znaky v našich regulárních výrazech stojí samy za sebe, ale tečka má občas speciální význam. Vyplatí se tedy regulární výraz převést z řetězce do nějaké podoby, kde bude u každého znaku zachyceno, jestli se před ním vyskytlo zpětné lomítko (a je tedy *escaped*) či ne, abychom s touto informací mohli později pracovat.

Operaci, při níž převádíme textovou reprezentaci dat na bohatší datovou strukturu, kde jsou explicitně zakódované různé další informace, se obecně říká **parsování**. Např. i tokenizaci textu, tj. převod z jednoho dlouhého textového řetězce na seznam textových řetězců, z něhož lze poznat hranice slov, bychom mohli nazvat parsováním.

V tomto případě chceme převést řetězec zachycující regulární výraz na seznam znaků regulárního výrazu a u každého z nich si podržet informaci, zda mu předcházelo zpětné lomítko. K zachycení každého znaku bychom mohli použít běžnou n-tici, např. tečku po zpětném lomítku bychom reprezentovali jako `(".", True)`, tečku bez předcházejícího zpětného lomítka jako `(".", False)`. Pro přehlednost je ale dobré si vytvořit nový typ objektu pomocí funkce `namedtuple()` (pojmenovaná n-tice):

In [4]:
from collections import namedtuple

RegexChar = namedtuple("RegexChar", "value escaped")

Takto jsme vytvořili nový typ objektu `RegexChar`. Instance tohoto objektu se vytváří pomocí konstruktoru `RegexChar()`, jemuž je potřeba předat dva argumenty, které pak budou odpovídat atributům `value` a `escaped`:

In [5]:
rchar = RegexChar("c", False)

Pojmenované n-tice (mezi něž náš nově vytvořený typ objektu `RegexChar` spadá) v mnohých ohledech fungují stejně jako běžné n-tice, můžeme z nich např. vytáhnout prvky pomocí číselné indexace:

In [6]:
rchar[0]

'c'

In [7]:
rchar[1]

False

Ale mají velkou výhodu v tom, že je u nich na první pohled mnohem jasnější, co reprezentují za data -- srovnejte běžnou n-tici (kde není úplně jasné, co má znamenat n-tice jako celek, ani co znamenají její jednotlivé položky)...

In [8]:
tuple(rchar)

('c', False)

... s pojmenovanou n-ticí:

In [9]:
rchar

RegexChar(value='c', escaped=False)

Navíc nabízí možnost odkazovat na jednotlivé položky pomocí jmen atributů, což je mnohem spolehlivější (u číselné indexace si člověk snadno splete pořadí):

In [10]:
rchar.value

'c'

In [11]:
rchar.escaped

False

Parsování tedy v našem případě znamená, že chceme převést řetězec reprezentující regulární výraz na seznam pojmenovaných n-tic typu `RegexChar`.

To zní asi trochu abstraktně, tak si raději ukažme příklad. Regulární výraz, který vyhledá písmeno `M`, pak jakýkoli znak a nakonec tečku vypadá jako řetězec takto: `M.\.`. Převeden na seznam objektů typu `RegexChar` by měl vypadat následovně:

```python
[
    RegexChar(value="M", escaped=False),
    # první tečce nepředcházelo zpětné lomítko, není tedy escaped
    # a má speciální význam "matchuj cokoli"
    RegexChar(value=".", escaped=False),
    # druhé tečce lomítko předcházelo, escaped tentokrát je a je to
    # běžná tečka
    RegexChar(value=".", escaped=True)
]
```

Doplňte implementaci funkce `parse_regex()`, která provede tuto operaci. Podmínky, které je potřeba splňovat, jsou naznačené pomocí testů v další buňce.

In [12]:
def parse_regex(pattern):
    ### BEGIN SOLUTION
    ans = []
    escaped = False
    for char in pattern:
        if char == "\\":
            escaped = True
        else:
            ans.append(RegexChar(char, escaped))
            escaped = False
    return ans
    ### END SOLUTION

In [13]:
# regulární výraz bez speciálních znaků
assert (
    parse_regex("abc")
    ==
    [
        RegexChar(value="a", escaped=False),
        RegexChar(value="b", escaped=False),
        RegexChar(value="c", escaped=False)
    ]
)
# regulární výraz s tečkou ve funkci speciálního znaku
assert (
    parse_regex("Mr.")
    ==
    [
        RegexChar(value="M", escaped=False),
        RegexChar(value="r", escaped=False),
        RegexChar(value=".", escaped=False)
    ]
)
# regulární výraz s tečkou ve funkci speciálního znaku
# a následně tečky (escaped=True)
assert (
    parse_regex("M.\.")
    ==
    [
        RegexChar(value="M", escaped=False),
        RegexChar(value=".", escaped=False),
        RegexChar(value=".", escaped=True)
    ]
)
# bez použití syntaxe raw string (r"...") z "\n" vznikne
# regulární výraz se znakem nového řádku
assert (
    parse_regex("\n")
    ==
    [RegexChar(value="\n", escaped=False)]
)
# s raw stringem získáme regulární výraz se znakem "n"
# s vlastností escaped=True
assert (
    parse_regex(r"\n")
    ==
    [RegexChar(value="n", escaped=True)]
)

## Matchování -- fáze 1

Nyní přikročíme k implementaci matchování. Nebudeme se ale rovnou snažit matchovat celý výraz vůči celému řetězci: abychom se v tom neutopili, začneme tím, že vyladíme matchování jednoho znaku regulárního výrazu vůči jednomu znaku z textu. Rozdělit tímto způsobem úlohu do více dílčích funkcí se vyplatí, můžeme je vyvíjet a zkoušet individuálně a složitou funkcionalitu budovat z ozkoušených jednodušších součástek.

Oproti tomu napsat rovnou jen jednu velkou funkci, která dělá všechno, je mnohem těžší, člověk často neví, kde začít. Rozčlenění úkolu na podúkoly nám totiž mimo jiné pomůže utřídit myšlenky, jak onen úkol vůbec řešit. Nemluvě o tom, že čím složitější a delší funkce s více účely, tím hůř se v ní hledají chyby.

Doplňte implementaci funkce `match_one()`, která namatchuje jeden znak regulárního výrazu a jeden znak z prohledávaného řetězce. Pracuje s následujícími argumenty:

- `regex_char`: část regulárního výrazu v podobě jednoho objektu typu `RegexChar`
- `text_char`: část prohledávaného textu v podobě řetězce o jednom znaku

Funkce vrací `True` nebo `False` podle toho, zda část textu části regulárního výrazu odpovídá či ne. Podmínky, které je potřeba splňovat, jsou naznačené pomocí testů v další buňce.

In [14]:
def match_one(regex_char, text_char):
    ### BEGIN SOLUTION
    if not regex_char.value:
        return True
    elif not text_char:
        return False
    elif regex_char.value == "." and not regex_char.escaped:
        return True
    return regex_char.value == text_char
    ### END SOLUTION

In [15]:
# výraz s konkrétním znakem matchuje jen a jen tento znak
assert match_one(RegexChar("a", False), "a")
assert not match_one(RegexChar("a", False), "z")
# tečka matchuje jakýkoli znak, ...
assert match_one(RegexChar(".", False), "a")
assert match_one(RegexChar(".", False), "z")
assert match_one(RegexChar(".", False), ".")
# ... pokud ovšem není escaped...
assert not match_one(RegexChar(".", True), "a")
assert not match_one(RegexChar(".", True), "z")
# ... (pak matchuje jen tečku)
assert match_one(RegexChar(".", True), ".")
# prázdný výraz matchuje jakýkoli znak, ...
assert match_one(RegexChar("", False), "z")
# ... naopak prázdný znak není matchován žádným výrazem, ...
assert not match_one(RegexChar("a", False), "")
# ... kromě prázdného
assert match_one(RegexChar("", False), "")

## Matchování -- fáze 2

Všechny dílčí součástky máme nyní k dispozici a můžeme konečně přikročit k implementaci plnohodnotného matchování.

**TIP:** Možná se vám bude hodit funkce `zip()`, která slouží k paralelnímu procházení více kolekcí najednou:

In [16]:
for num, char in zip([1, 2, 3], "abc"):
    print(f"{num}: {char}")

1: a
2: b
3: c


Doplňte implementaci funkce `match()` s následujícími argumenty:

- `pattern`: hledaný regulární výraz
- `text`: prohledávaný text

Funkce vrací `True`, pokud výraz matchuje (na začátku textu, ne uprostřed), jinak `False`. Podmínky, které je potřeba splňovat, jsou naznačené pomocí testů v další buňce. Při implementaci funkce je **nutné použít dříve definované funkce** `match_one()` a `parse_regex()`.

In [17]:
def match(pattern, text):
    ### BEGIN SOLUTION
    return all(match_one(p, c) for (p, c) in zip(parse_regex(pattern), text))
    ### END SOLUTION

In [18]:
assert match("...", "abc")
assert not match("abc", "...")
assert match("M.\.", "Ms.")
assert match("M.\.", "Mr.")
assert not match("M.\.", "Mrs")
assert match("t.p", "tip")
assert match("t.p", "typ")
# prohledávaný text může být delší než regulární výraz...
assert match("t.p", "typologie")
# ... ale aby to byl match, musí být hledaný vzorec
# na začátku textu
assert not match("t.p", "prototyp")
# speciálních teček může být pochopitelně víc za sebou
# na různých místech ve výrazu
assert match("v..k..", "vlákat")
assert match("v..k..", "vankúš")

## Závěr

Říká se, že prubířským kamenem programátorské zdatnosti je implementace vlastního programovacího jazyka. Jednoduchý dotazovací jazyk, který jste vytvořili výše, je takovým prvním krůčkem v tomto směru, namístě jsou tedy gratulace :)

I když v praxi (jako většina programátorů) zřejmě nikdy žádný programovací jazyk vytvářet nebudete, je to užitečné cvičení. Zejména s parsováním textu (= převodem textu do strukturovaných dat) se jako lingvisti či obecně vzato humanitně zaměření odborníci setkáte velmi často.

Pro odvážné a zvídavé přikládám [odkaz na článek](https://nickdrane.com/build-your-own-regex/), který byl pro tento úkol inspirací. Autor jde dál a implementuje např. operátory `?` a `*`. I tak na to stačí možná až překvapivě málo kódu. Jen upozorním, že implementace není v Pythonu, ale v JavaScriptu.